In [14]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import pandas as pd
import requests
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import MWETokenizer

In [15]:
#--------------------------------------- TO BE REMOVED ---------------------------------------
def clean_page_to_parse(page_string):
    page_string = re.sub('<!-- WWW Designer Jeff Anderson janderson@iisd.ca --!>','',str(page_string))
    page_string = re.sub('<!-- WWW Designer Jeff Anderson janderson@iisd.ca --!>','',str(page_string))
    page_string = re.sub(r'<HTML>',r'<html>',str(page_string))
    page_string = re.sub(r'</HTML>',r'</html>',str(page_string))

    return page_string

#Extract the <p> tag from a specific html link 
def extract_p_tags(html_link):
    page = urllib.request.urlopen(html_link).read()
    page = clean_page_to_parse(page)
    soup = BeautifulSoup(page,'html.parser')
    list_tp = soup.find_all('p')
    return list_tp

#Clean the sentence by removing special char
def clean_tp(sentence):
    s = re.sub("\r\n\s\s+"," ",sentence)
    s = re.sub("\r\n","",s)
    s = re.sub("\s\s+"," ",s)
    s = re.sub("\r\n","",s)
    p = re.compile(r'<.*?>')
    return p.sub('', s)


In [16]:
def extract_from_paragraphes_sentences(list_paragraphes):
    list_paragraphes_foot = []
    #Remove footer
    for i in range(len(list_paragraphes)):
        if('Sustaining Donors' not in list_paragraphes[i] and 'This issue of' not in list_paragraphes[i]):
            list_paragraphes_foot.append(list_paragraphes[i])
    
    #Split into sentence from paragraph
    splited_sentence = []
    for s in list_paragraphes_foot:
        splited_sentence += sent_tokenize(s)
    return splited_sentence

In [17]:
def open_list_meetings():
    f = open('list_meetings.csv')
    return csv.reader(f)

In [18]:
def extract_tuple(line):
    l = line.replace('"',"")
    l = l.replace('\n',"")
    l = l.split('\t')
    return l


In [19]:
#Extract complete dataset of Paula, put it into list of tuples
def extract_dataset():
    list_data = list(open('statements_count.csv','r'))
    list_dataset = []
    for line in list_data:
        list_dataset.append(extract_tuple(line))
    return list_dataset


In [20]:
#Extract all the entities mentioned in Paula's dataset
def extract_entities():
    list_dataset = extract_dataset()
    set_entities = set()
    for line in list_dataset:
        set_entities.add(line[0])
    return set_entities

In [21]:
def count_occurences(list_sentences, dict_occ, tokenizer, list_entities, meetings):

    for s in list_sentences:
        #Split line into words with tokenizer to detetc entity
        line = s.replace(",","")
        line_splited = word_tokenize(line)
        tokens = tokenizer.tokenize(line_splited)
        
        for entity in list_entities:
            #Increment value of intervention of the entity
            if(entity in tokens):
                dict_occ[entity] += 1
    rows = [(meetings[0],entity,meetings[4],dict_occ[entity],meetings[5]) for entity in dict_occ]
    return rows
    

                



In [22]:
def extract_occurences_issue_ENB():
    #List meetings named 'ENB'
    list_ENBs = open_list_meetings()
    #list_ENBs = [('COP','13','2007-12-03','Bali, Indonesia','349','2007-12-10','https://enb.iisd.org/vol12/enb12349e.html','Issue')]
    tokenizer = MWETokenizer(extract_entities(), separator=' ')
    list_entities = extract_entities()
    occurences_meetings = []
    k=0
    #2 Count number time entities mentioned
    for m in list(list_ENBs)[80:]:
        k +=1

        if(k==20):
            break
        
        request = requests.get(m[6])
        
        if(request.status_code == 200):
            list_paragraphes = [clean_tp(str(line)) for line in extract_p_tags(m[6])]
            list_sentences = extract_from_paragraphes_sentences(list_paragraphes)
            dict_occurences = dict.fromkeys(extract_entities(), 0)
            occurences_meetings += count_occurences(list_sentences, dict_occurences, tokenizer, list_entities, m)

    
    with open('list_occurences.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        #header
        writer.writerow(('meeting_type','entity','issue_number','interventions','date'))
        writer.writerows(occurences_meetings)






In [23]:
extract_occurences_issue_ENB()